# best

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
from numba import jit,njit

test_num = 14

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

&nbsp;

## 입력 shape 및 형태 정의 함수

In [2]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [3]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [4]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [5]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=30)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [6]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 0):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=3,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[128],
    mlp_dropout=0.25,
    dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:1'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [7]:
@jit
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

@jit
def y_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(14, len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values - df.iloc[index-1])

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [8]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1000
batch = 64

In [9]:

def feature_engineer(df):
       #시간 코사인 인코딩
       if df['datadate'].all() == 0:
         df['datadate'] = 20180520
       df['datetime'] = pd.to_datetime(df['datadate'], format="%Y%m%d")
       df['cos_time'] = np.cos(2*np.pi*df['datetime'].dt.dayofyear/365)

       # 결측치
       df = df.replace(' ', np.nan)
       df = df.fillna(0)

       # 주산지 초기온도, 최대온도, 최저온도, 평균온도, 강수량, 습도
       df['주산지_초기온도'] = (df['주산지_0_초기온도(℃)'] + df['주산지_1_초기온도(℃)'] + df['주산지_2_초기온도(℃)'])/3
       df['주산지_최대온도'] = (df['주산지_0_최대온도(℃)'] + df['주산지_1_최대온도(℃)'] + df['주산지_2_최대온도(℃)'])/3
       df['주산지_최저온도'] = (df['주산지_0_최저온도(℃)'] + df['주산지_1_최저온도(℃)'] + df['주산지_2_최저온도(℃)'])/3
       df['주산지_평균온도'] = (df['주산지_0_평균온도(℃)'] + df['주산지_1_평균온도(℃)'] + df['주산지_2_평균온도(℃)'])/3
       df['주산지_강수량'] = (df['주산지_0_강수량(ml)'] + df['주산지_1_강수량(ml)'] + df['주산지_2_강수량(ml)'])/3

       # 필요한 컬럼 선택
       market_cols = ['단가(원)', '경매건수', '거래량', '거래대금(원)'] # 

       market_code_cols = [] # '도매시장코드', '도매법인코드', '산지코드 '

       market_statistics_cols = [] 
       # , '하위가격 거래물량(kg)', '상위가격 거래물량(kg)', '일자별_도매가격_최대(원)', '하위가격 평균가(원)',
       #  '상위가격 평균가(원)', '해당일자_전체거래물량(kg)', '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소(원)'
       #  '일자별_도매가격_평균(원)', '일자별_도매가격_최소(원)', , 
       
       export_cols = [] # '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)', '무역수지(달러)'
       
       date_cols = ['datadate', 'cos_time'] # ,, '일자구분_중순', '일자구분_초순', '일자구분_하순', '월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월','월구분_3월', '월구분_4월', '월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월', 'datetime'

       weather_cols = [] 
      #  '주산지_0_초기온도(℃)', '주산지_1_초기온도(℃)', '주산지_2_초기온도(℃)',
      #  '주산지_0_최대온도(℃)', '주산지_1_최대온도(℃)', '주산지_2_최대온도(℃)',
      #  '주산지_0_최저온도(℃)', '주산지_1_최저온도(℃)', '주산지_2_최저온도(℃)',
      #  '주산지_0_평균온도(℃)', '주산지_1_평균온도(℃)', '주산지_2_평균온도(℃)',
      #  '주산지_0_강수량(ml)', '주산지_1_강수량(ml)', '주산지_2_강수량(ml)',
      #  '주산지_0_습도(%)', '주산지_1_습도(%)', '주산지_2_습도(%)'

       selected_columns = ['해당일자_전체평균가격(원)']  + market_cols + market_code_cols + market_statistics_cols + export_cols +  date_cols + weather_cols # train 에서 사용하지 않는 열

       df = df[[c for c in df.columns if c in selected_columns]]

       return df

&nbsp;

## Train 과정

In [10]:
from sklearn.preprocessing import MinMaxScaler

losses = []
scalers = {}

for i in tqdm(data_list):
    print(i)
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    # 사용할 열 선택 및 index 설정
    df = feature_engineer(df)
    df.set_index('datadate', drop=True, inplace=True)
    
    # nan 처리
    df = df.replace({' ': np.nan})
    df = df.fillna(0)
    
    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']
    scaler = MinMaxScaler()
    x = pd.DataFrame(scaler.fit_transform(x))
    scalers[df_number] = scaler

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    # y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = y_time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y
    
    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch, learning_rate = 0.001, verbose = 0)
    
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')
    loss = transformer.evaluate(astype_data(x_val), y_val, verbose=0)
    print('\nvalid_loss: ', loss)
    losses.append(loss)

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

  0%|          | 0/37 [00:00<?, ?it/s]

./aT_data/data/train/train_4.csv
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-09-21 13:03:15.753398: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-21 13:03:15.753808: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-09-21 13:03:16.697154: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-21 13:03:16.856661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 13:03:17.156944: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  3%|▎         | 1/37 [01:24<50:54, 84.84s/it]


valid_loss:  1408579.75
./aT_data/data/train/train_15.csv


2022-09-21 13:04:40.319119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 13:04:40.499639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  5%|▌         | 2/37 [02:48<48:58, 83.96s/it]

...


valid_loss:  130541.2109375
./aT_data/data/train/train_18.csv


2022-09-21 14:20:24.149875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:20:24.407856: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
 97%|█████████▋| 36/37 [1:18:33<02:31, 151.75s/it]


valid_loss:  101500592.0
./aT_data/data/train/train_9.csv


2022-09-21 14:21:49.382943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:21:49.644349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 37/37 [1:19:59<00:00, 129.71s/it]


valid_loss:  332148.0625


In [11]:
losses

[1408579.75,
 557766.125,
 315245.0625,
 73800.7890625,
 27782410.0,
 5546303.0,
 4960280.0,
 2640655.5,
 2004114.625,
 70874512.0,
 1421587.375,
 999465.5625,
 1168714.75,
 2262624.75,
 269160.40625,
 362445.6875,
 4391882.5,
 4977432.5,
 4186075.0,
 823748.5625,
 308031.875,
 1168825.25,
 7058630.5,
 561510.4375,
 212205.296875,
 2918951.25,
 3084290.0,
 9059943.0,
 11501902.0,
 4349948.0,
 881660.4375,
 4649194.5,
 4826007.0,
 341928.21875,
 130541.2109375,
 101500592.0,
 332148.0625]

&nbsp;

## Test 과정

In [12]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)

        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택 및 index 설정

        df = feature_engineer(df)
        df = df[[c for c in df.columns if c != '해당일자_전체평균가격(원)']]

        df.set_index('datadate', drop=True, inplace=True)
        
        # # nan 처리
        # df = df.replace({' ': np.nan})
        # df = df.fillna(0)

        # # train input 과 형상 맞추기
        # add_col = [i for i in check_col if i not in df.columns]

        # for a in add_col:
        #     df[a] = 0

        # # ' ' -> nan 으로 변경
        # for a in df.columns:
        #     df[a] = df[a].replace({' ': np.nan})

        # # nan 처리
        # df = df.fillna(0)
        # # df = df.interpolate(method = 'values')

        df = pd.DataFrame(scalers[file_number].transform(df))

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists(f'./model_output/{test_num}') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]2022-09-21 14:23:15.136861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 40ms/step


2022-09-21 14:23:15.509757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:23:15.610908: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 32ms/step


2022-09-21 14:23:15.716871: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:23:15.826836: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:23:15.914412: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-21 14:23:47.290673: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:23:47.377801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:23:47.465130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 31ms/step


2022-09-21 14:23:47.553463: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:23:47.636885: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:23:47.724450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 30ms/step


2022-09-21 14:23:47.810283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 14:23:47.896233: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 10/10 [00:33<00:00,  3.35s/it]


&nbsp;

## 정답 제출 파일생성

In [13]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  9.59it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [14]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.12500,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,-693.15710,-19.097841,-377.399930,141.716770,-88.644280,-2506.050300,11.875956,-645.432800,-245.990840,-286.774500,...,-832.677550,-67.378296,-254.831160,-90.530550,-772.159060,-739.136660,-2836.573200,-1077.593800,-266.935900,259.654970
2,-532.95660,-38.389650,-1422.012600,146.744580,-62.948715,-2465.993200,-3.146761,-738.629400,-269.118380,-231.466740,...,-968.301450,-99.839485,-286.303340,-163.199980,-840.205440,-733.489000,-2727.923800,-1249.543100,-288.679470,366.844000
3,-661.29156,-29.544336,-243.223600,133.070590,-72.422930,-2489.325000,-25.628496,-740.656800,-283.941560,-286.925540,...,-1096.354000,-118.346000,-336.055600,-248.582280,-697.189330,-639.804140,-3011.941200,-1327.768300,-304.290040,207.997830
4,-482.32327,-24.093884,-728.907400,136.818020,-57.031280,-2463.820600,-14.073060,-747.216200,-227.937330,-140.657360,...,-1072.414400,-121.599525,-204.546800,-176.583970,-690.810600,-616.085100,-2927.601800,-1329.749900,-268.038200,486.687560
5,-628.60340,-5.585401,1032.584400,132.068680,-50.492558,-2481.788000,1.658677,-738.745500,-199.908040,-186.661030,...,-963.008540,-99.210380,-6.601847,-39.045887,-755.449160,-701.524230,-2733.353800,-1288.381200,-284.327420,206.295490
6,-674.17596,-12.762246,-17.208794,136.143260,-72.508840,-2565.562000,-24.613108,-665.523440,-189.538970,-131.633870,...,-840.728940,-88.441550,151.606400,-127.395134,-632.447900,-532.636200,-2735.159200,-1124.289900,-284.659850,535.398900
7,-122.77166,-10.103426,-8.256965,17.318916,4.289736,-302.584750,-16.811434,-372.848850,-23.837465,-16.360197,...,-302.393620,-27.383186,-15.232314,-25.672796,-66.540115,-67.223740,-214.452940,-351.601840,-64.226690,64.397285
8,-688.91980,-25.845020,-351.915160,122.637780,-85.331760,-2562.314000,-4.284051,-698.348630,-237.804120,-284.080840,...,-952.654500,-80.197130,-265.319370,-108.070990,-692.364800,-746.489600,-2800.131600,-1229.852000,-278.442140,290.384640
9,-515.07100,-45.351536,-1278.289200,130.882050,-66.424545,-2406.207800,-14.829288,-767.558170,-275.281980,-229.853520,...,-1076.467700,-111.079990,-272.454250,-174.656190,-734.993100,-731.831900,-2625.595500,-1329.575700,-296.993740,383.647100


- 변동률 계산 

In [15]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']

for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값
    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1])/base_d)
    
    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean())/base_d
    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [16]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [17]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,-0.179058,-0.014021,-0.129700,0.041680,-0.022454,-0.270809,0.004371,-0.192034,-0.050081,-0.244476,...,-0.230530,-0.057800,-0.212513,-0.135504,-0.504801,-0.206016,-0.091885,0.124375,0,d+1
1,-0.137675,-0.028184,-0.488700,0.043159,-0.015945,-0.266480,-0.001158,-0.219763,-0.054789,-0.197326,...,-0.259001,-0.104196,-0.231241,-0.134469,-0.485466,-0.238890,-0.099370,0.175719,0,d+2
2,-0.170827,-0.021690,-0.083588,0.039138,-0.018345,-0.269001,-0.009432,-0.220366,-0.057807,-0.244604,...,-0.304009,-0.158709,-0.191880,-0.117294,-0.536010,-0.253845,-0.104743,0.099631,0,d+3
3,-0.124595,-0.017689,-0.250502,0.040240,-0.014446,-0.266245,-0.005179,-0.222318,-0.046405,-0.119911,...,-0.185041,-0.112741,-0.190124,-0.112946,-0.521001,-0.254224,-0.092265,0.233124,0,d+4
4,-0.162383,-0.004101,0.354866,0.038843,-0.012790,-0.268187,0.000610,-0.219797,-0.040699,-0.159129,...,-0.005972,-0.024929,-0.207914,-0.128609,-0.486432,-0.246315,-0.097872,0.098816,0,d+5
5,-0.174155,-0.009369,-0.005914,0.040041,-0.018367,-0.277240,-0.009058,-0.198012,-0.038588,-0.112218,...,0.137149,-0.081336,-0.174062,-0.097647,-0.486753,-0.214944,-0.097986,0.256457,0,d+6
6,-0.031715,-0.007417,-0.002838,0.005094,0.001087,-0.032698,-0.006187,-0.110933,-0.004853,-0.013947,...,-0.013780,-0.016391,-0.018313,-0.012324,-0.038164,-0.067220,-0.022108,0.030846,0,d+7
7,-0.177964,-0.018974,-0.120942,0.036069,-0.021615,-0.276889,-0.001577,-0.207778,-0.048414,-0.242179,...,-0.240018,-0.068999,-0.190552,-0.136852,-0.498316,-0.235125,-0.095846,0.139095,0,d+8
8,-0.133055,-0.033295,-0.439307,0.038494,-0.016826,-0.260020,-0.005457,-0.228370,-0.056044,-0.195950,...,-0.246473,-0.111511,-0.202284,-0.134165,-0.467255,-0.254191,-0.102232,0.183768,0,d+9
9,-0.159071,-0.030165,-0.061893,0.032317,-0.016809,-0.261615,-0.011296,-0.218027,-0.058275,-0.234791,...,-0.299065,-0.162479,-0.157816,-0.116226,-0.517856,-0.266871,-0.105771,0.100202,0,d+10


- 정답 양식으로 변경

In [18]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [19]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.179058,-0.014021,-0.129700,0.041680,-0.022454,-0.270809,0.004371,-0.192034,...,-0.096366,-0.111923,-0.230530,-0.057800,-0.212513,-0.135504,-0.504801,-0.206016,-0.091885,0.124375
1,0,d+2,-0.137675,-0.028184,-0.488700,0.043159,-0.015945,-0.266480,-0.001158,-0.219763,...,-0.112061,-0.165845,-0.259001,-0.104196,-0.231241,-0.134469,-0.485466,-0.238890,-0.099370,0.175719
2,0,d+3,-0.170827,-0.021690,-0.083588,0.039138,-0.018345,-0.269001,-0.009432,-0.220366,...,-0.126881,-0.196586,-0.304009,-0.158709,-0.191880,-0.117294,-0.536010,-0.253845,-0.104743,0.099631
3,0,d+4,-0.124595,-0.017689,-0.250502,0.040240,-0.014446,-0.266245,-0.005179,-0.222318,...,-0.124110,-0.201991,-0.185041,-0.112741,-0.190124,-0.112946,-0.521001,-0.254224,-0.092265,0.233124
4,0,d+5,-0.162383,-0.004101,0.354866,0.038843,-0.012790,-0.268187,0.000610,-0.219797,...,-0.111449,-0.164800,-0.005972,-0.024929,-0.207914,-0.128609,-0.486432,-0.246315,-0.097872,0.098816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.017169,-0.090603,-0.111864,-0.147927,0.075078,0.276018,-0.003979,NaN,...,0.100819,0.604327,-0.193293,-0.058679,-0.169903,0.220468,-0.070613,-0.019520,-0.353669,-0.163980
146,9,d+12,-0.024144,-0.094108,-0.306029,-0.135757,0.064877,0.288518,-0.003099,NaN,...,0.088553,0.536768,-0.037298,-0.122545,-0.188561,0.239831,-0.085954,-0.022283,-0.350705,-0.149029
147,9,d+13,-0.026520,-0.079615,-0.288377,-0.123810,0.061390,0.282827,-0.005296,NaN,...,0.067761,0.283808,-0.053329,-0.056917,-0.161215,0.204126,-0.064020,-0.007540,-0.354855,-0.189833
148,9,d+14,-0.003816,-0.016804,-0.043378,-0.048815,0.027020,0.032157,-0.004542,NaN,...,0.017109,0.166281,-0.011387,-0.011001,-0.019464,0.016681,0.017832,-0.011578,-0.092571,-0.030373


- 제출 양식

In [20]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [21]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.179058,-0.014021,-0.129700,0.041680,-0.022454,-0.270809,0.004371,-0.192034,...,-0.096366,-0.111923,-0.230530,-0.057800,-0.212513,-0.135504,-0.504801,-0.206016,-0.091885,0.124375
1,0,d+2,-0.137675,-0.028184,-0.488700,0.043159,-0.015945,-0.266480,NaN,-0.219763,...,-0.112061,-0.165845,-0.259001,-0.104196,-0.231241,-0.134469,-0.485466,-0.238890,-0.099370,0.175719
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,-0.009432,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,-0.124595,-0.017689,-0.250502,0.040240,-0.014446,-0.266245,NaN,-0.222318,...,-0.124110,-0.201991,-0.185041,-0.112741,-0.190124,-0.112946,-0.521001,-0.254224,-0.092265,0.233124
4,0,d+5,-0.162383,-0.004101,0.354866,0.038843,-0.012790,-0.268187,NaN,-0.219797,...,-0.111449,-0.164800,-0.005972,-0.024929,-0.207914,-0.128609,-0.486432,-0.246315,-0.097872,0.098816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.017169,-0.090603,-0.111864,-0.147927,0.075078,0.276018,-0.003979,NaN,...,0.100819,0.604327,-0.193293,-0.058679,-0.169903,0.220468,-0.070613,-0.019520,-0.353669,-0.163980
146,9,d+12,-0.024144,-0.094108,-0.306029,-0.135757,0.064877,0.288518,-0.003099,NaN,...,0.088553,0.536768,-0.037298,-0.122545,-0.188561,0.239831,-0.085954,-0.022283,-0.350705,-0.149029
147,9,d+13,-0.026520,-0.079615,-0.288377,-0.123810,0.061390,0.282827,-0.005296,NaN,...,0.067761,0.283808,-0.053329,-0.056917,-0.161215,0.204126,-0.064020,-0.007540,-0.354855,-0.189833
148,9,d+14,-0.003816,-0.016804,-0.043378,-0.048815,0.027020,0.032157,-0.004542,NaN,...,0.017109,0.166281,-0.011387,-0.011001,-0.019464,0.016681,0.017832,-0.011578,-0.092571,-0.030373
